In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# "utils" 폴더에 있는 match 함수를 기술한 match.py를 import
from utils.match import match

In [ ]:
class MultiBoxLoss(nn.Module):
    def __init__(self, jaccard_thresh=0.5, neg_pos=3, device='cpu'):
        super(MultiBoxLoss, self).__init__()
        self.jaccard_thresh = jaccard_thresh  # jaccard ths
        self.negpos_ratio = neg_pos  # 3:1 Hard Negative Mining (neg/pos 비율)
        self.device = device 

    def forward(self, predictions, targets):
        """
        손실 함수 계산

        Parameters
        ----------
        predictions : SSD net의 훈련시의 출력(tuple)
            (loc=torch.Size([num_batch, 8732, 4]), conf=torch.Size([num_batch, 8732, 21]), dbox_list=torch.Size [8732, 4])

        targets : [num_batch, num_objs, 5]
            5는 정답의 어노테이션 정보[xmin, ymin, xmax, ymax, label_ind]를 나타낸다

        Returns
        -------
        loss_l : 텐서
            loc의 손실값
        loss_c : 텐서
            conf의 손실값
        """
        loc_data, conf_data, dbox_list = predictions

        num_batch = loc_data.size(0) 
        num_dbox = loc_data.size(1)  
        num_classes = conf_data.size(2)  
        
        # 손실 함수에 사용되는 변수
        conf_t_label = torch.LongTensor(num_batch, num_dbox).to(self.device)  # 개별 DBox에 가장 가까운 정답 BBox의 클래스 저장
        loc_t = torch.Tensor(num_batch, num_dbox, 4).to(self.device)  # 개별 DBox에 가장 가까운 정답 BBox의 좌표 저장

        for idx in range(num_batch):
            truths = targets[idx][:, :-1].to(self.device)  # 정답 BBox
            labels = targets[idx][:, -1].to(self.device)

            dbox = dbox_list.to(self.device)

            ## match 실행
            # loc_t : 개별 DBox에 가장 가까운 정답 BBox의 위치 정보 저장
            # conf_t_label : 개별 DBox에 가장 가까운 정답 BBox의 클래스 라벨 저장 (가장 가까운 BBox와의 jaccard overlap이 ths 미만인 경우 0)
            variance = [0.1, 0.2]
            match(self.jaccard_thresh, truths, dbox,
                  variance, labels, loc_t, conf_t_label, idx)
            
        ## loss 계산
        # 1. loss_l : offset 정보에 대한 손실
        pos_mask = conf_t_label > 0  # positive DBox에서만 계산함
        pos_idx = pos_mask.unsqueeze(pos_mask.dim()).expand_as(loc_data)

        loc_p = loc_data[pos_idx].view(-1, 4)  # positive DBox 위치 정보
        loc_t = loc_t[pos_idx].view(-1, 4)  # 정답 BBox 위치 정보

        loss_l = F.smooth_l1_loss(loc_p, loc_t, reduction='sum')

        # 2. loss_c : 클래스 예측 손실 - cross entropy (Hard Negative Mining 적용)
        batch_conf = conf_data.view(-1, num_classes)
        loss_c = F.cross_entropy(batch_conf, conf_t_label.view(-1), reduction='none')

        # Hard Neagative Mining
        num_pos = pos_mask.long().sum(1, keepdim=True)  # 배치별 객체에 속하는 클래스에 대한 예측의 수
        loss_c = loss_c.view(num_batch, -1)  # torch.Size([num_batch, 8732])
        loss_c[pos_mask] = 0  # 객체를 탐지한 DBox의 손실은 0

        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)

        num_neg = torch.clamp(num_pos*self.negpos_ratio, max=num_dbox)  # 배경(0)에 속하는 DBox의 수 
        neg_mask = idx_rank < (num_neg).expand_as(idx_rank)  # torch.Size([num_batch, 8732])
        
        # Positive DBox의 신뢰도를 추출하는 mask / Hard Negative Mining으로 추출된 Negative DBox의 신뢰도를 추출하는 mask
        pos_idx_mask = pos_mask.unsqueeze(2).expand_as(conf_data)  # pos_mask: torch.Size([num_batch, 8732]) -> pos_idx_mask: torch.Size([num_batch, 8732, 21])
        neg_idx_mask = neg_mask.unsqueeze(2).expand_as(conf_data)
        
        conf_hnm = conf_data[(pos_idx_mask+neg_idx_mask).gt(0)].view(-1, num_classes)  # torch.Size([num_pos+num_neg, 21])
        conf_t_label_hnm = conf_t_label[(pos_mask+neg_mask).gt(0)]

        loss_c = F.cross_entropy(conf_hnm, conf_t_label_hnm, reduction='sum')

        N = num_pos.sum()
        loss_l /= N
        loss_c /= N

        return loss_l, loss_c